In [10]:
import pandas as pd

dfDblp_a = pd.read_csv('dblp_acm_exp_data/exp_data/tableA.csv', sep=',', index_col=0, low_memory=False)
dfAcm = pd.read_csv('dblp_acm_exp_data/exp_data/tableB.csv', sep=',', index_col=0, low_memory=False)

df_train_acm = pd.read_csv('dblp_acm_exp_data/exp_data/train.csv', sep=',', low_memory=False)
df_test_acm = pd.read_csv('dblp_acm_exp_data/exp_data/test.csv', sep=',', low_memory=False)
df_valid_acm = pd.read_csv('dblp_acm_exp_data/exp_data/valid.csv', sep=',', low_memory=False)


dfDblp_b = pd.read_csv('dblp_scholar_exp_data/exp_data/tableA.csv', sep=',', index_col=0, low_memory=False)
dfScholar = pd.read_csv('dblp_scholar_exp_data/exp_data/tableB.csv', sep=',', index_col=0, low_memory=False)

df_train_scholar = pd.read_csv('dblp_scholar_exp_data/exp_data/train.csv', sep=',', low_memory=False)
df_test_scholar = pd.read_csv('dblp_scholar_exp_data/exp_data/test.csv', sep=',', low_memory=False)
df_valid_scholar = pd.read_csv('dblp_scholar_exp_data/exp_data/valid.csv', sep=',', low_memory=False)

In [11]:
dfDblp_a = dfDblp_a.drop_duplicates(subset=['title', 'authors', 'venue', 'year'], keep='first')
dfAcm = dfAcm.drop_duplicates(subset=['title', 'authors', 'venue', 'year'], keep='first')

dfDblp_b = dfDblp_b.drop_duplicates(subset=['title', 'authors', 'venue', 'year'], keep='first')
dfScholar = dfScholar.drop_duplicates(subset=['title', 'authors', 'venue', 'year'], keep='first')

In [ ]:
def rename_index( df, name): 
    df_new = pd.DataFrame()
    for index, row in df.iterrows():
        row['uri'] = name + str(index)
        df_new = df_new.append(row, ignore_index=True, sort = True)
    return df_new

dfDblp_a = rename_index(dfDblp_a, 'dblp_a_')
dfAcm = rename_index(dfAcm, 'acm_')

dfDblp_b = rename_index(dfDblp_b, 'dblp_b_')
dfScholar = rename_index(dfScholar, 'scholar_')

display(dfAcm)

In [12]:
def rename_validation_data( df, nameL, nameR):
    df_new = pd.DataFrame()
    for index, row in df.iterrows():
        row['ltable_id'] = nameL + str(row['ltable_id'])
        row['rtable_id'] = nameR + str(row['rtable_id'])
        if(row['label'] == 1 ):
            row['label'] = 'TRUE'
        else:
            row['label'] = 'FALSE'
        df_new = df_new.append(row, ignore_index=True, sort = True)
        cols = df_new.columns.tolist()
        cols = cols[1:] + cols[:1]
        df_new = df_new[cols]
    return df_new


df_train_acm_new = rename_validation_data(df_train_acm, 'dblp_a_', 'acm_')
df_test_acm_new = rename_validation_data(df_test_acm, 'dblp_a_', 'acm_')
df_valid_acm_new = rename_validation_data(df_valid_acm, 'dblp_a_', 'acm_')

df_train_scholar_new = rename_validation_data(df_train_scholar, 'dblp_b_', 'scholar_')
df_test_scholar_new = rename_validation_data(df_test_scholar, 'dblp_b_', 'scholar_')
df_valid_scholar_new = rename_validation_data(df_valid_scholar, 'dblp_b_', 'scholar_')

In [ ]:
dfCompleteGoldstandard = df_train_acm_new.append(df_test_acm_new)
dfCompleteGoldstandard = dfCompleteGoldstandard.append(df_valid_acm_new)
dfCompleteGoldstandard = dfCompleteGoldstandard.append(df_train_scholar_new)
dfCompleteGoldstandard = dfCompleteGoldstandard.append(df_test_scholar_new)
dfCompleteGoldstandard = dfCompleteGoldstandard.append(df_valid_scholar_new)
dfCompleteGoldstandard.to_csv('goldstandard_citation_complete.csv', sep=',', encoding='utf-8', index=False)

In [ ]:
#Strip true values for goldstandard
#display(df_train_acm_new)
dfGSCiteACM_true = df_train_acm_new[df_train_acm_new['label'] == 'TRUE'].append(df_test_acm_new[df_test_acm_new['label'] == 'TRUE'])
dfGSCiteACM_true = dfGSCiteACM_true.append(df_valid_acm_new[df_valid_acm_new['label'] == 'TRUE'])

dfGSCiteScholar_true = df_train_scholar_new[df_train_scholar_new['label'] == 'TRUE'].append(df_test_scholar_new[df_test_scholar_new['label'] == 'TRUE'])
dfGSCiteScholar_true = dfGSCiteScholar_true.append(df_valid_scholar_new[df_valid_scholar_new['label'] == 'TRUE'])

In [ ]:
display(len(dfGSCiteScholar_true))
#Deduplicate Goldstandard
dfGSCiteScholar_true = dfGSCiteScholar_true.sort_values(by=['rtable_id'])
dfGSCiteScholar_true_reduced = dfGSCiteScholar_true.drop_duplicates(subset=['ltable_id'], keep='first')
listExclude_gs_scholar = [str(row['rtable_id']) for index, row in dfGSCiteScholar_true.iterrows() if row['rtable_id'] not in dfGSCiteScholar_true_reduced['rtable_id'].values]
dfGSCiteScholar_true_reduced = dfGSCiteScholar_true_reduced[~dfGSCiteScholar_true_reduced['rtable_id'].isin(listExclude_gs_scholar)]

dfGSCiteScholar_true_reduced = dfGSCiteScholar_true_reduced.sort_values(by=['ltable_id'])
dfGSCiteScholar_true_reduced = dfGSCiteScholar_true_reduced.drop_duplicates(subset=['rtable_id'], keep='first')
listExclude_gs_dblp = [str(row['ltable_id']) for index, row in dfGSCiteScholar_true.iterrows() if row['ltable_id'] not in dfGSCiteScholar_true_reduced['ltable_id'].values]
dfGSCiteScholar_true_reduced = dfGSCiteScholar_true_reduced[~dfGSCiteScholar_true_reduced['ltable_id'].isin(listExclude_gs_dblp)]
display(len(dfGSCiteScholar_true_reduced))

#Find removed values right handside
display(len(dfDblp_b))
listExclude_dblp = [str(row['ltable_id']) for index, row in dfGSCiteScholar_true.iterrows() if row['ltable_id'] not in dfGSCiteScholar_true_reduced['ltable_id'].values]
dfDblp_bNew = dfDblp_b[~dfDblp_b['uri'].isin(listExclude_dblp)]
display(len(dfDblp_bNew))

#Find removed values left handside
display(len(dfScholar))
listExclude_scholar = [str(row['rtable_id']) for index, row in dfGSCiteScholar_true.iterrows() if row['rtable_id'] not in dfGSCiteScholar_true_reduced['rtable_id'].values]
dfScholarNew = dfScholar[~dfScholar['uri'].isin(listExclude_scholar)]
display(len(dfScholarNew))

In [ ]:
display(listExclude_scholar)

In [ ]:
cols = dfGSCiteScholar_true_reduced.columns.tolist()
cols = cols[1:] + cols[:1]
dfGSCiteScholar_true_reduced = dfGSCiteScholar_true_reduced[cols]

dfGSCiteACM_true.to_csv('goldstandard_citation_true_acm.csv', sep=',', encoding='utf-8', index=False, header=None)
dfGSCiteScholar_true_reduced.to_csv('goldstandard_citation_true_scholar.csv', sep=',', encoding='utf-8',index=False, header=None)

In [1]:
import numpy as np
import pandas as pd

dfDblp = pd.read_csv('dblp.csv', sep=',')
dfAcm = pd.read_csv('acm.csv', sep=',')
dfScholar = pd.read_csv('scholar.csv', sep=',')

#dfDblp = dfDblp.replace('?', np.nan)
#dfAcm = dfAcm.replace('?', np.nan)
#dfScholar = dfScholar.replace('?', np.nan)

#dfDblp = dfDblp.replace('nan', np.nan)
#dfAcm = dfAcm.replace('nan', np.nan)
#dfScholar = dfScholar.replace('nan', np.nan)

dfAllCitations = dfDblp.append(dfAcm)
dfAllCitations = dfAllCitations.append(dfScholar)

C:\Users\Alex\Anaconda3\envs\py36\lib\site-packages\pandas\core\frame.py:6211: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


In [ ]:
dfAllCitations = dfDblp_a.append(dfAcm)
dfAllCitations = dfAllCitations.append(dfDblp_bNew)
dfAllCitations = dfAllCitations.append(dfScholarNew)

dfAllCitations = dfAllCitations.replace('?', np.nan)
dfAllCitations = dfAllCitations.replace('nan', np.nan)
dfAllCitations = dfAllCitations.replace('', np.nan)

In [2]:
# seperate most frequent words
import nltk
from nltk.probability import FreqDist
from nltk import bigrams

#nltk.download('stopwords')
stopset = set(nltk.corpus.stopwords.words('english'))
tokens = nltk.WordPunctTokenizer().tokenize(' '.join(dfAllCitations['title'].values))
clean = [token.lower() for token in tokens if token.lower() not in stopset and token.isalpha()]

numbers_of_words = len(clean)
freq = FreqDist(clean)

dfFreq = pd.DataFrame.from_dict(freq, orient='index')
dfFreq['word'] = dfFreq.index
dfFreq['percentage'] = dfFreq[0] / numbers_of_words

# Take frequent words as they are considered to be stop words
dfFreqInt = dfFreq[dfFreq['percentage'] > 0.004]
frequent_words_titel = dfFreqInt['word'].values.tolist()

In [ ]:
display(dfFreq.sort_values(by=[0], ascending=False))

In [ ]:
display(dfAllCitations.describe(include='all'))

#display(dfAllCitations)

In [3]:
def determine_freqent_words(hotel_name):
    tokens = nltk.WordPunctTokenizer().tokenize(hotel_name)
    clean = [token.lower() for token in tokens if token.lower() not in stopset and token.isalpha()]
    return ' '.join(set([token for token in clean if token in frequent_words_titel]))

def determine_infreqent_words(hotel_name):
    tokens = nltk.WordPunctTokenizer().tokenize(hotel_name)
    clean = [token.lower() for token in tokens if token.lower() not in stopset and token.isalpha()]
    return ' '.join(set([token for token in clean if token not in frequent_words_titel]))

dfAllCitations['frequent_part_title'] = dfAllCitations['title'].apply(determine_freqent_words)
dfAllCitations['infrequent_part_title'] = dfAllCitations['title'].apply(determine_infreqent_words)
dfAllCitations = dfAllCitations.replace('', np.nan)
dfAllCitations = dfAllCitations.replace(' ', np.nan)

In [4]:
import numpy as np
#dfAllCitations.replace('NaN', np.nan)
#dfAllCitations['year'] = dfAllCitations['year'].astype('int64')
dfScholar = dfAllCitations[dfAllCitations['uri'].str.contains('scholar')]
dfAcm = dfAllCitations[dfAllCitations['uri'].str.contains('acm')]
dfDblp = dfAllCitations[dfAllCitations['uri'].str.contains('dblp')]

In [9]:
import csv
dfScholar.to_csv('scholar.csv', sep=',', encoding='utf-8', index=False, quotechar='"', quoting=csv.QUOTE_ALL, float_format='%.0f')
dfAcm.to_csv('acm.csv', sep=',', encoding='utf-8', index=False, quotechar='"', quoting=csv.QUOTE_ALL, float_format='%.0f')
dfDblp.to_csv('dblp.csv', sep=',', encoding='utf-8', index=False, quotechar='"', quoting=csv.QUOTE_ALL, float_format='%.0f')

In [8]:
display(dfScholarNew)

,authors,frequent_part_title,infrequent_part_title,title,uri,venue,year
0,qd inc,NaN,road sorrento valley,11578 sorrento valley road,scholar_0,"san diego ,",NaN
1,"as argon , jg hannoosh",NaN,initiation crazes polystyrene,initiation of crazes in polystyrene,scholar_1,"phil . mag ,",NaN
2,"gh hansen , ll wetterberg , h sjã ¶ strã ¶ m ,...",NaN,immunogold demonstrated aminopeptidase n metho...,immunogold labelling is a quantitative method ...,scholar_2,"the histochemical journal ,",1992.0
3,"tm hammett , p harmon , w rhodes",NaN,disease among burden releasees facilities inma...,the burden of infectious disease among inmates...,scholar_3,see,NaN
4,jr cogdell,NaN,faculty science engineering role advising,the role of faculty advising in science and en...,scholar_4,"new directions for teaching and learning ,",1995.0
5,wm schmidt,NaN,multiplicity linear recurrence sequences zero,the zero multiplicity of linear recurrence seq...,scholar_5,to,NaN
6,ra haats,NaN,kindergarten difficulties reading screeners yo...,predicitve validity of kindergarten screeners ...,scholar_6,NaN,2002.0
7,"jr norris , j deisenhofer",NaN,reaction photosynthetic center,the photosynthetic reaction center,scholar_7,"san diego : academic ,",NaN
8,f bennour,NaN,windows aw f sous ndiaye hachage scalable aire...,"f. , diene , aw , ndiaye , y. hachage linã © a...",scholar_8,"res . rep. ceria , u. paris ,",NaN
9,"i borg , jc lingoes",NaN,multidimensional structure similarity analysis,multidimensional similarity structure analysis,scholar_9,NaN,1987.0
